## Imports

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras

import matplotlib.pyplot as plt
import numpy as np

import urllib.request
import zipfile
from IPython import display

## Utilities

In [ ]:
def plot_results(images, n_cols=None):
    '''visualizes fake images'''
    display.clear_output(wait=False)

    n_cols = n_cols or len(images)
    n_rows = (len(images) - 1) // n_cols + 1

    if images.shape[-1] == 1:
        images = np.squeeze(images, axis=-1)

    plt.figure(figsize=(n_cols, n_rows))

    for index, image in enumerate(images):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(image, cmap="binary")
        plt.axis("off")

## Get the training data



In [ ]:
# download the dataset
training_url = "https://storage.googleapis.com/tensorflow-1-public/tensorflow-3-temp/signs-training.zip"
training_file_name = "signs-training.zip"
urllib.request.urlretrieve(training_url, training_file_name)

# extract to local directory
training_dir = "/tmp"
zip_ref = zipfile.ZipFile(training_file_name, 'r')
zip_ref.extractall(training_dir)
zip_ref.close()

## Preprocess the images



In [ ]:
BATCH_SIZE = 32

# mapping function for preprocessing the image files
def map_images(file):
  '''converts the images to floats and normalizes the pixel values'''
  img = tf.io.decode_png(tf.io.read_file(file))
  img = tf.dtypes.cast(img, tf.float32)
  img = img / 255.0

  return img

# create training batches
filename_dataset = tf.data.Dataset.list_files("/tmp/signs-training/*.png")
image_dataset = filename_dataset.map(map_images).batch(BATCH_SIZE)

## Build the generator



In [ ]:
# You'll pass the random_normal_dimensions to the first dense layer of the generator
random_normal_dimensions = 32

generator = keras.models.Sequential([
   tf.keras.layers.Dense(7*7*128, input_shape = [random_normal_dimensions]),
   tf.keras.layers.Reshape([7,7,128]),
   tf.keras.layers.BatchNormalization(),
   tf.keras.layers.Conv2DTranspose(64,5,strides = 2,padding = 'same',
                                   activation = 'selu'),
   tf.keras.layers.BatchNormalization(),
   tf.keras.layers.Conv2DTranspose(1,5,strides = 2,padding = 'same',
                                   activation = 'tanh')
])

## Build the discriminator


In [ ]:
### START CODE HERE ###
discriminator = keras.models.Sequential([
  tf.keras.layers.Conv2D(64,5,strides = 2,
                         padding = 'same',
                         activation = tf.keras.layers.LeakyReLU(0.2)),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Conv2D(128,5,strides = 2,
                         padding = 'same',
                         activation = tf.keras.layers.LeakyReLU(0.2)),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1,activation = 'sigmoid')
])

## Compile the discriminator



In [ ]:
discriminator.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop')
# Set discriminator.trainable to False
discriminator.trainable = False

## Build and compile the GAN model



In [ ]:
gan = keras.models.Sequential([generator, discriminator])
gan.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop')

## Train the GAN



In [ ]:
def train_gan(gan, dataset, random_normal_dimensions, n_epochs=50):

    # get the two sub networks from the GAN model
    generator, discriminator = gan.layers

    for epoch in range(n_epochs):
        print("Epoch {}/{}".format(epoch + 1, n_epochs))
        for real_images in dataset:

            # infer batch size from the current batch of real images
            real_batch_size = real_images.shape[0]

            # Train the discriminator - PHASE 1
            # Create the noise
            noise = tf.random.normal(shape = [real_batch_size,random_normal_dimensions])

            # Use the noise to generate fake images
            fake_images = generator(noise)

            # Create a list by concatenating the fake images with the real ones
            mixed_images = tf.concat([fake_images,real_images],axis = 0)

            # Create the labels for the discriminator
            # 0 for the fake images
            # 1 for the real images
            discriminator_labels = tf.constant(
                [[0.]] * real_batch_size + [[1.]] * real_batch_size
            )

            # Ensure that the discriminator is trainable
            discriminator.trainable = True

            # Use train_on_batch to train the discriminator with the mixed images and the discriminator labels
            discriminator.train_on_batch(mixed_images, discriminator_labels)

            # Train the generator - PHASE 2
            # create a batch of noise input to feed to the GAN
            noise = tf.random.normal([real_batch_size, random_normal_dimensions])

            # label all generated images to be "real"
            generator_labels = tf.constant([[1.]] * real_batch_size)

            # Freeze the discriminator
            discriminator.trainable = False

            # Train the GAN on the noise with the labels all set to be true
            gan.train_on_batch(noise, generator_labels)

        plot_results(fake_images, 16)
        plt.show()
    return fake_images

### Run the training



In [ ]:
# you can adjust the number of epochs
EPOCHS = 60

# run the training loop and collect images
fake_images = train_gan(gan, image_dataset, random_normal_dimensions, EPOCHS)

## Choose your best images to submit for grading!

Please visually inspect your 31 generated hand images.  They are indexed from 0 to 30, from left to right on the first row on top, and then continuing from left to right on the second row below it.

- Choose 16 images that you think look most like actual hands.
- Use the `append_to_grading_images()` function, pass in `fake_images` and a list of the indices for the 16 images that you choose to submit for grading (e.g. `append_to_grading_images(fake_images, [1, 4, 5, 6, 8... until you have 16 elements])`).

In [ ]:
# helper function to collect the images
def append_to_grading_images(images, indexes):
  l = []
  for index in indexes:
    if len(l) >= 16:
      print("The list is full")
      break
    l.append(tf.squeeze(images[index:(index+1),...], axis=0))
  l = tf.convert_to_tensor(l)
  return l

Please fill in the empty list (2nd parameter) with 16 indices indicating the images you want to submit to the grader.

In [ ]:
grading_images = append_to_grading_images(fake_images, [ ])

## Zip your selected images for grading

Please run the code below. This will save the images you chose to a zip file named `my-signs.zip`.

- Please download this file from the Files explorer on the left.
- Please return to the Coursera classroom and upload the zip file for grading.

In [ ]:
from PIL import Image
from zipfile import ZipFile

denormalized_images = grading_images * 255
denormalized_images = tf.dtypes.cast(denormalized_images, dtype = tf.uint8)

file_paths = []

for this_image in range(0,16):
  i = tf.reshape(denormalized_images[this_image], [28,28])
  im = Image.fromarray(i.numpy())
  im = im.convert("L")
  filename = "hand" + str(this_image) + ".png"
  file_paths.append(filename)
  im.save(filename)

with ZipFile('my-signs.zip', 'w') as zip:
  for file in file_paths:
    zip.write(file)